In [2]:
#import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import os
from sklearn.metrics import confusion_matrix

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col, when, count, lit, regexp_replace
import pyspark.sql.functions as f
from pyspark.sql.types import DoubleType
from pyspark.ml import Pipeline  
from pyspark.ml.feature import *  
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import RandomForestRegressor
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from sklearn.metrics import confusion_matrix
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier


spark= SparkSession.builder.getOrCreate()
infile = 'H1B_combined_final.csv'

spark = SparkSession \
    .builder \
    .appName("immi") \
    .getOrCreate()

In [4]:
df = spark.read.csv(infile, inferSchema=True, header = True)

In [5]:
df.show(10)

+------------------+-------------------+-------------+-------------+----------+--------------------+----------+--------------------+------------------+--------------------+-------------+--------------+---------------------+----------------+---------------+--------------+
|       CASE_NUMBER|        CASE_STATUS|RECEIVED_DATE|DECISION_DATE|VISA_CLASS|           JOB_TITLE|  SOC_CODE|           SOC_TITLE|FULL_TIME_POSITION|       EMPLOYER_NAME|EMPLOYER_CITY|EMPLOYER_STATE|WAGE_RATE_OF_PAY_FROM|WAGE_UNIT_OF_PAY|PREVAILING_WAGE|PW_UNIT_OF_PAY|
+------------------+-------------------+-------------+-------------+----------+--------------------+----------+--------------------+------------------+--------------------+-------------+--------------+---------------------+----------------+---------------+--------------+
|I-200-12240-490687|CERTIFIED-WITHDRAWN|   2012-08-27|   2015-10-16|      H-1B|POSTDOCTORAL RESE...|10-1021.00|BIOCHEMISTS AND B...|              null|UNIVERSITY OF MIC...|    ANN ARBO

In [6]:
emp_grp = df.groupBy("EMPLOYER_NAME").count()
emp_grp.show(truncate=False)

+-------------------------------------------------+-----+
|EMPLOYER_NAME                                    |count|
+-------------------------------------------------+-----+
|FIVEPALLS, INC                                   |3    |
|TERRATHERM, INC                                  |1    |
|SHIPPAN RACQUET CLUB                             |1    |
|VAN PARYS ARCHITECTURE AND DESIGN LLP            |1    |
|LANDED, INC.                                     |1    |
|ADEQUALITY, INC.                                 |4    |
|PIRAMAL CRITICAL CARE, INC.                      |9    |
|LOOKBOOKER, INC.                                 |1    |
|C1X INC                                          |2    |
|BAXTER HEALTHCARE CORPORATION                    |69   |
|STELLA & DOT, LLC                                |1    |
|NEWGEN SOFTWARE INC.                             |71   |
|NEW IEM POWER SYSTEMS, LLC, DBA IEM POWER SYSTEMS|4    |
|ANIMOTO INC.                                     |13   |
|MY BLUEBONNET

In [7]:
emp_grp.count()

233575

In [8]:
#Prevailing wage is in different units(yearly,monthly,hourly etc)
df.groupBy("PW_UNIT_OF_PAY").count().show()

+--------------+-------+
|PW_UNIT_OF_PAY|  count|
+--------------+-------+
|          Year|2791861|
|     Bi-Weekly|    362|
|         Month|   1628|
|          Hour| 187659|
|          null|   2157|
|          Week|    600|
|       61360.0|      1|
|       82867.0|      1|
+--------------+-------+



In [9]:
#Actual wage is in different units(yearly,monthly,hourly etc)
df.groupBy("WAGE_UNIT_OF_PAY").count().show()

+----------------+-------+
|WAGE_UNIT_OF_PAY|  count|
+----------------+-------+
|            Year|2792985|
|       Bi-Weekly|    656|
|           Month|   2521|
|            Hour| 187078|
|            null|     74|
|        150000.0|      1|
|         70000.0|      1|
|         88275.0|      1|
|            Week|    950|
|        155106.0|      1|
|              MA|      1|
+----------------+-------+



In [10]:
#Finding NULL columns in the selected features
df_Columns=["SOC_CODE","EMPLOYER_NAME","EMPLOYER_STATE","WAGE_RATE_OF_PAY_FROM","PW_UNIT_OF_PAY"]
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+--------+-------------+--------------+---------------------+--------------+
|SOC_CODE|EMPLOYER_NAME|EMPLOYER_STATE|WAGE_RATE_OF_PAY_FROM|PW_UNIT_OF_PAY|
+--------+-------------+--------------+---------------------+--------------+
|      21|          138|           178|                   43|          2157|
+--------+-------------+--------------+---------------------+--------------+



In [11]:
#Copnverting unit of prevailing wages into a single unit,which is monthly
df = df.withColumn("pw_pay", when((df.PW_UNIT_OF_PAY == 'Bi-Weekly'), lit(df.PREVAILING_WAGE * 26))
              .when((df.PW_UNIT_OF_PAY == 'Month'), lit(df.PREVAILING_WAGE * 12))
             .when((df.PW_UNIT_OF_PAY == 'Hour'), lit(df.PREVAILING_WAGE * 40 * 52))
             .when((df.PW_UNIT_OF_PAY == 'Week'), lit(df.PREVAILING_WAGE * 52))
             .otherwise(lit(df.PREVAILING_WAGE)))

In [12]:
#Copnverting unit of actual wages into a single unit,which is monthly
df = df.withColumn("pay", when((df.WAGE_UNIT_OF_PAY == 'Bi-Weekly'), lit(df.WAGE_RATE_OF_PAY_FROM * 26))
              .when((df.WAGE_UNIT_OF_PAY == 'Month'), lit(df.WAGE_RATE_OF_PAY_FROM * 12))
             .when((df.WAGE_UNIT_OF_PAY == 'Hour'), lit(df.WAGE_RATE_OF_PAY_FROM * 40 * 52))
             .when((df.WAGE_UNIT_OF_PAY == 'Week'), lit(df.WAGE_RATE_OF_PAY_FROM * 52))
             .otherwise(lit(df.WAGE_RATE_OF_PAY_FROM)))

In [13]:
feats = ["SOC_CODE", 
         "EMPLOYER_NAME", 
         "EMPLOYER_STATE", 
         "pay",
        "pw_pay"]

# subset the dataframe on these predictors
df1 = df.select(*feats,'CASE_STATUS')
df1.show(10)

+----------+--------------------+--------------+------------------+------------------+-------------------+
|  SOC_CODE|       EMPLOYER_NAME|EMPLOYER_STATE|               pay|            pw_pay|        CASE_STATUS|
+----------+--------------------+--------------+------------------+------------------+-------------------+
|10-1021.00|UNIVERSITY OF MIC...|            MI|           41000.0|           36067.0|CERTIFIED-WITHDRAWN|
|   11-1011|GOODMAN NETWORKS,...|            TX|          400000.0|          242674.0|CERTIFIED-WITHDRAWN|
|   11-1011|PORTS AMERICA GRO...|            NJ|          264000.0|          193066.0|CERTIFIED-WITHDRAWN|
|   11-1011|GATES CORPORATION...|            CO|          220314.0|          220314.0|CERTIFIED-WITHDRAWN|
|   11-1011|PEABODY INVESTMEN...|            MO|356990.39999999997|157518.40000000002|          WITHDRAWN|
|   11-1011|BURGER KING CORPO...|            FL|          225000.0|          225000.0|CERTIFIED-WITHDRAWN|
|   11-1011|BT AND MK ENERGY ...|    

In [14]:
#Finding NULL columns in the selected features
df_Columns=["SOC_CODE","EMPLOYER_NAME","EMPLOYER_STATE","pw_pay","pay","CASE_STATUS"]
df1.select([count(when(col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+--------+-------------+--------------+------+---+-----------+
|SOC_CODE|EMPLOYER_NAME|EMPLOYER_STATE|pw_pay|pay|CASE_STATUS|
+--------+-------------+--------------+------+---+-----------+
|      21|          138|           178|  2008| 43|          0|
+--------+-------------+--------------+------+---+-----------+



In [15]:
#dropped the null columns
df1=df1.na.drop()

In [16]:
#Verifying if there are NULL columns
df_Columns=["SOC_CODE","EMPLOYER_NAME","EMPLOYER_STATE","pw_pay","pay","CASE_STATUS"]
df1.select([count(when(col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+--------+-------------+--------------+------+---+-----------+
|SOC_CODE|EMPLOYER_NAME|EMPLOYER_STATE|pw_pay|pay|CASE_STATUS|
+--------+-------------+--------------+------+---+-----------+
|       0|            0|             0|     0|  0|          0|
+--------+-------------+--------------+------+---+-----------+



In [17]:
#Case status needs cleaning. Some values are in caps and some are in smalls
df1.groupBy("CASE_STATUS").count().show()

+--------------------+-------+
|         CASE_STATUS|  count|
+--------------------+-------+
|              Denied|   6674|
|           CERTIFIED|1694655|
| CERTIFIED-WITHDRAWN| 105125|
|Certified - Withd...|  31448|
|           Withdrawn|  20608|
|           Certified|1036456|
|           WITHDRAWN|  60779|
|              DENIED|  26283|
+--------------------+-------+



In [18]:
#Changing all vaues to small letters
df1 = df1.withColumn("CASE_STATUS",f.lower(f.col("CASE_STATUS")))
df1.show(5)

+----------+--------------------+--------------+------------------+------------------+-------------------+
|  SOC_CODE|       EMPLOYER_NAME|EMPLOYER_STATE|               pay|            pw_pay|        CASE_STATUS|
+----------+--------------------+--------------+------------------+------------------+-------------------+
|10-1021.00|UNIVERSITY OF MIC...|            MI|           41000.0|           36067.0|certified-withdrawn|
|   11-1011|GOODMAN NETWORKS,...|            TX|          400000.0|          242674.0|certified-withdrawn|
|   11-1011|PORTS AMERICA GRO...|            NJ|          264000.0|          193066.0|certified-withdrawn|
|   11-1011|GATES CORPORATION...|            CO|          220314.0|          220314.0|certified-withdrawn|
|   11-1011|PEABODY INVESTMEN...|            MO|356990.39999999997|157518.40000000002|          withdrawn|
+----------+--------------------+--------------+------------------+------------------+-------------------+
only showing top 5 rows



In [19]:
#Needs further cleaning 
df1.groupBy("CASE_STATUS").count().show()

+--------------------+-------+
|         CASE_STATUS|  count|
+--------------------+-------+
|              denied|  32957|
|           withdrawn|  81387|
| certified-withdrawn| 105125|
|certified - withd...|  31448|
|           certified|2731111|
+--------------------+-------+



In [20]:
# Changing all certified-withdrawn cases to "certified"
df2 = df1.withColumn('CASE_STATUS', when(df1.CASE_STATUS == 'certified-withdrawn',regexp_replace(df1.CASE_STATUS,'certified-withdrawn','certified'))
                     .when(df1.CASE_STATUS == 'certified - withdrawn',regexp_replace(df1.CASE_STATUS,'certified - withdrawn','certified'))
                     .otherwise(df1.CASE_STATUS))

In [21]:
# Target variable is clean now with 3 classes
df2.groupBy("CASE_STATUS").count().show()

+-----------+-------+
|CASE_STATUS|  count|
+-----------+-------+
|     denied|  32957|
|  withdrawn|  81387|
|  certified|2867684|
+-----------+-------+



In [22]:
#adding label feature for the 3 classes
df2 = df2.withColumn("label", when((df2.CASE_STATUS == 'denied'), 0)
                    .when((df2.CASE_STATUS == 'certified'), 1)
                     .otherwise(2))

In [23]:
df2.show(5)

+----------+--------------------+--------------+------------------+------------------+-----------+-----+
|  SOC_CODE|       EMPLOYER_NAME|EMPLOYER_STATE|               pay|            pw_pay|CASE_STATUS|label|
+----------+--------------------+--------------+------------------+------------------+-----------+-----+
|10-1021.00|UNIVERSITY OF MIC...|            MI|           41000.0|           36067.0|  certified|    1|
|   11-1011|GOODMAN NETWORKS,...|            TX|          400000.0|          242674.0|  certified|    1|
|   11-1011|PORTS AMERICA GRO...|            NJ|          264000.0|          193066.0|  certified|    1|
|   11-1011|GATES CORPORATION...|            CO|          220314.0|          220314.0|  certified|    1|
|   11-1011|PEABODY INVESTMEN...|            MO|356990.39999999997|157518.40000000002|  withdrawn|    2|
+----------+--------------------+--------------+------------------+------------------+-----------+-----+
only showing top 5 rows



In [24]:
df2.groupBy("label").count().show()

+-----+-------+
|label|  count|
+-----+-------+
|    1|2867684|
|    2|  81387|
|    0|  32957|
+-----+-------+



In [25]:
df2.dtypes

[('SOC_CODE', 'string'),
 ('EMPLOYER_NAME', 'string'),
 ('EMPLOYER_STATE', 'string'),
 ('pay', 'string'),
 ('pw_pay', 'string'),
 ('CASE_STATUS', 'string'),
 ('label', 'int')]

In [26]:
#Changing the feature "pay" from string to doubletype
df2=df2.withColumn("pay",df.pay.cast(DoubleType()))

In [27]:
#Changing the feature "pw_pay" from string to doubletype
df2=df2.withColumn("pw_pay",df.pw_pay.cast(DoubleType()))

In [28]:
df2.dtypes

[('SOC_CODE', 'string'),
 ('EMPLOYER_NAME', 'string'),
 ('EMPLOYER_STATE', 'string'),
 ('pay', 'double'),
 ('pw_pay', 'double'),
 ('CASE_STATUS', 'string'),
 ('label', 'int')]

In [29]:
#Variable with the features selected to build the model
ftrs = ["EMPLOYER_STATE",
        "SOC_CODE",
        "pw_pay",
         "pay"]

# subset the dataframe on these predictors
df3 = df2.select(*ftrs,'label')
df3.show(10)

+--------------+----------+------------------+------------------+-----+
|EMPLOYER_STATE|  SOC_CODE|            pw_pay|               pay|label|
+--------------+----------+------------------+------------------+-----+
|            MI|10-1021.00|           36067.0|           41000.0|    1|
|            TX|   11-1011|          242674.0|          400000.0|    1|
|            NJ|   11-1011|          193066.0|          264000.0|    1|
|            CO|   11-1011|          220314.0|          220314.0|    1|
|            MO|   11-1011|157518.40000000002|356990.39999999997|    2|
|            FL|   11-1011|          225000.0|          225000.0|    1|
|            FL|   11-1011|           91021.0|          120000.0|    1|
|            CA|   11-1011|          150000.0|          150000.0|    1|
|            TN|   11-1011|          127546.0|          175000.0|    1|
|            VA|   11-1011|          154648.0|          155000.0|    2|
+--------------+----------+------------------+------------------

In [30]:
#Split data into 80% training and 20% test data
seed = 314
train_test = [0.8, 0.2]
training, test = df3.randomSplit(train_test, seed=seed)

In [36]:
# Reusable Function with preprocessing, model build and evaluation steps 
def model(ev):
   #Feature Preprocessing steps
   categoricalCols = [field for (field, dataType) in training.dtypes if dataType == "string"]
   indexOutputCols = [x + "Index" for x in categoricalCols]
   oheOutputCols = [x + "OHE" for x in categoricalCols]
   stringIndexer = StringIndexer(inputCols=categoricalCols,outputCols=indexOutputCols,handleInvalid="skip")
   oheEncoder = OneHotEncoder(inputCols=indexOutputCols,outputCols=oheOutputCols)
   numericCols = [field for (field, dataType) in training.dtypes if ((dataType == "double") & (field != "label"))]
   assemblerInputs = oheOutputCols + numericCols
   va = VectorAssembler(inputCols=assemblerInputs, outputCol="features", handleInvalid="skip") 

   # Model Build using pipelines
   pipeline = Pipeline(stages=[stringIndexer,oheEncoder,va, ev])
   model = pipeline.fit(training)
    
   #Model Evaluation using MulticlassClassificationEvaluator and printing stats for the model
   preds = model.transform(test)
   mcEvaluator = MulticlassClassificationEvaluator(metricName="accuracy")
   acc = mcEvaluator.evaluate(preds)
   predictionAndLabels = model.transform(test).select('label', 'prediction')
   metrics = MulticlassMetrics(predictionAndLabels.rdd.map(lambda x: tuple(map(float, x))))
   confusion_matrix = metrics.confusionMatrix().toArray()
   labels = [int(l) for l in metrics.call('labels')]
   f1 = metrics.fMeasure(1.0)
   precision = metrics.precision(1.0)
   recall = metrics.recall(1.0)
   data = metrics.confusionMatrix().toArray()
   rdd1 = spark.sparkContext.parallelize(data)
   rdd2 = rdd1.map(lambda x: [int(i) for i in x])
   df = rdd2.toDF()
   print("Printing Confusion Matrix\n")
   df.show()
   return f1, precision, recall, acc 
   
   
   

In [37]:
#Logistic regression model
ev = LogisticRegression(labelCol='label', featuresCol='features',maxIter=10)
lr_f1, lr_precision, lr_recall, lr_acc = model(ev)
print("Printing stats for Logistic classification model\n")
print("Accuracy: " + str(lr_acc))
print("F1: " + str(lr_f1))
print("Precision: " + str(lr_precision))
print("Recall: " + str(lr_recall))

Printing Confusion Matrix

+----+------+-----+
|  _1|    _2|   _3|
+----+------+-----+
| 115|    82|   16|
|6440|573829|16115|
|   3|    14|   73|
+----+------+-----+

Printing stats for Logistic classification model

Accuracy: 0.9620068813297424
F1: 0.9806452825706715
Precision: 0.9998327307575031
Recall: 0.9621804072543865


In [38]:
#Decision Tree classifier model
ev = DecisionTreeClassifier(featuresCol="features", labelCol="label")
dt_f1, dt_precision, dt_recall, dt_acc = model(ev)
print("Printing stats for Decision Tree classifier model\n")
print("Accuracy: " + str(dt_acc))
print("F1: " + str(dt_f1))
print("Precision: " + str(dt_precision))
print("Recall: " + str(dt_recall))

Printing Confusion Matrix

+----+------+-----+
|  _1|    _2|   _3|
+----+------+-----+
| 110|     4|   15|
|6446|573920|16182|
|   2|     1|    7|
+----+------+-----+

Printing stats for Decision Tree classifier model

Accuracy: 0.9620403997405675
F1: 0.9806633728415778
Precision: 0.9999912880602866
Recall: 0.962068433722014


In [39]:
# Random Classifier model
ev = RandomForestClassifier(labelCol="label",featuresCol="features", numTrees=10)
rf_f1, rf_precision, rf_recall, rf_acc = model(ev)
print("Printing stats for Random Forest classifier model\n")
print("Accuracy: " + str(rf_acc))
print("F1: " + str(rf_f1))
print("Precision: " + str(rf_precision))
print("Recall: " + str(rf_recall))

Printing Confusion Matrix

+------+
|    _1|
+------+
|573925|
+------+

Printing stats for Random Forest classifier model

Accuracy: 0.9618526966399469
F1: 0.9805554701301541
Precision: 1.0
Recall: 0.9618526966399469


In [31]:
# Reusable Function with preprocessing, model build and evaluation steps for cross validation
def cvmodel(ev,paramGrid):
   #Feature Preprocessing steps
   categoricalCols = [field for (field, dataType) in training.dtypes if dataType == "string"]
   indexOutputCols = [x + "Index" for x in categoricalCols]
   oheOutputCols = [x + "OHE" for x in categoricalCols]
   stringIndexer = StringIndexer(inputCols=categoricalCols,outputCols=indexOutputCols,handleInvalid="skip")
   oheEncoder = OneHotEncoder(inputCols=indexOutputCols,outputCols=oheOutputCols)
   numericCols = [field for (field, dataType) in training.dtypes if ((dataType == "double") & (field != "label"))]
   assemblerInputs = oheOutputCols + numericCols
   va = VectorAssembler(inputCols=assemblerInputs, outputCol="features", handleInvalid="skip") 
   
   # Model Build using pipelines
   pipeline = Pipeline(stages=[stringIndexer,oheEncoder,va, ev])
   crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)

   model = crossval.fit(training)
    
   #Model Evaluation using MulticlassClassificationEvaluator and printing stats for the model
   preds = model.transform(test)
   mcEvaluator = MulticlassClassificationEvaluator(metricName="accuracy")
   acc = mcEvaluator.evaluate(preds)
   predictionAndLabels = model.transform(test).select('label', 'prediction')
   metrics = MulticlassMetrics(predictionAndLabels.rdd.map(lambda x: tuple(map(float, x))))
   confusion_matrix = metrics.confusionMatrix().toArray()
   labels = [int(l) for l in metrics.call('labels')]
   f1 = metrics.fMeasure(1.0)
   precision = metrics.precision(1.0)
   recall = metrics.recall(1.0)
   data = metrics.confusionMatrix().toArray()
   rdd1 = spark.sparkContext.parallelize(data)
   rdd2 = rdd1.map(lambda x: [int(i) for i in x])
   df = rdd2.toDF()
   print("Printing Confusion Matrix\n")
   df.show()
   return f1, precision, recall, acc 

In [32]:
# Cross validation for the Logistic Classifier model
ev = LogisticRegression(labelCol='label', featuresCol='features',maxIter=10) #Evaluator for the model

# Create ParamGrid for cross validation
paramGrid = (ParamGridBuilder()
             .addGrid(ev.regParam, [0.1, 1.0, 2.0]) # Regularization parameter
             .addGrid(ev.elasticNetParam, [0.0, 0.5, 1.0]) #Elastic Net Paramater (Ridge = 0)
             .addGrid(ev.maxIter, [1, 5, 10]) #Number of Iterations
             .build())
cv_lr_f1, cv_lr_precision, cv_lr_recall, cv_lr_acc = cvmodel(ev,paramGrid)
print("Printing stats for CV Logistic classification model\n")
print("Accuracy: " + str(cv_lr_acc))
print("F1: " + str(cv_lr_f1))
print("Precision: " + str(cv_lr_precision))
print("Recall: " + str(cv_lr_recall))

Printing Confusion Matrix

+----+------+-----+
|  _1|    _2|   _3|
+----+------+-----+
|  85|    59|   11|
|6472|573852|16120|
|   1|    14|   73|
+----+------+-----+

Printing stats for CV Logistic classification model

Accuracy: 0.9619951498859536
F1: 0.9806343127680245
Precision: 0.9998728056801847
Recall: 0.9621221774382842


In [33]:
# Cross validation for the Decision Tree Classifier model
ev = DecisionTreeClassifier(featuresCol="features", labelCol="label")  #Evaluator for the model

# Create ParamGrid for cross validation
paramGrid = (ParamGridBuilder()\
            .addGrid(ev.maxDepth, [2, 5, 10]) # number of splits for each tree
            .addGrid(ev.maxBins, [5, 10 , 20]) # Number of bins
            .build())
cv_dt_f1, cv_dt_precision, cv_dt_recall, cv_dt_acc = cvmodel(ev,paramGrid)
print("Printing stats for CV Decision Tree classifier model\n")
print("Accuracy: " + str(cv_dt_acc))
print("F1: " + str(cv_dt_f1))
print("Precision: " + str(cv_dt_precision))
print("Recall: " + str(cv_dt_recall))

Printing Confusion Matrix

+----+------+-----+
|  _1|    _2|   _3|
+----+------+-----+
| 162|    16|   26|
|6388|573900|16129|
|   8|     9|   49|
+----+------+-----+

Printing stats for CV Decision Tree classifier model

Accuracy: 0.9621644178606203
F1: 0.9807389634824694
Precision: 0.9999564403014332
Recall: 0.9622462136391149


In [34]:
# Cross validation for the RF model
ev = RandomForestClassifier()
paramGrid = (ParamGridBuilder()\
            .addGrid(ev.maxDepth, [2, 5, 10]) # number of splits for each tree
            .addGrid(ev.maxBins, [5, 10 , 20]) # Number of bins
            .addGrid(ev.numTrees,[5, 20, 50]) # Number of decision trees
            .build())

cv_rf_f1, cv_rf_precision, cv_rf_recall, cv_rf_acc = cvmodel(ev,paramGrid)
print("Printing stats for CV Random Forest classifier model\n")
print("Accuracy: " + str(cv_rf_acc))
print("F1: " + str(cv_rf_f1))
print("Precision: " + str(cv_rf_precision))
print("Recall: " + str(cv_rf_recall))

Printing Confusion Matrix

+------+
|    _1|
+------+
|573925|
+------+

Printing stats for CV Random Forest classifier model

Accuracy: 0.9618526966399469
F1: 0.9805554701301541
Precision: 1.0
Recall: 0.9618526966399469


In [40]:
# Create dataframe from the results of different models built above
columns = ["Model","Accuracy","F1 Score", "Precision", "Recall"]
data = [("Logistic Regression", lr_acc, lr_f1, lr_precision, lr_recall), 
        ('CV Logistic Regression', cv_lr_acc, cv_lr_f1, cv_lr_precision, cv_lr_recall), 
        ('Decision Tree Classfier', dt_acc, dt_f1, dt_precision, dt_recall), 
        ('CV Decision Tree Classfier', cv_dt_acc, cv_dt_f1, cv_dt_precision, cv_dt_recall),
        ('Random Forest Classifier', rf_acc, rf_f1, rf_precision, rf_recall), 
        ('CV Random Forest Classifier', cv_rf_acc, cv_rf_f1, cv_rf_precision, cv_rf_recall)]
rdd = spark.sparkContext.parallelize(data)
acc_df = spark.createDataFrame(rdd).toDF(*columns)
acc_df.show(truncate=False)

+---------------------------+------------------+------------------+------------------+------------------+
|Model                      |Accuracy          |F1 Score          |Precision         |Recall            |
+---------------------------+------------------+------------------+------------------+------------------+
|Logistic Regression        |0.9620068813297424|0.9806452825706715|0.9998327307575031|0.9621804072543865|
|CV Logistic Regression     |0.9619951498859536|0.9806343127680245|0.9998728056801847|0.9621221774382842|
|Decision Tree Classfier    |0.9620403997405675|0.9806633728415778|0.9999912880602866|0.962068433722014 |
|CV Decision Tree Classfier |0.9621644178606203|0.9807389634824694|0.9999564403014332|0.9622462136391149|
|Random Forest Classifier   |0.9618526966399469|0.9805554701301541|1.0               |0.9618526966399469|
|CV Random Forest Classifier|0.9618526966399469|0.9805554701301541|1.0               |0.9618526966399469|
+---------------------------+-----------------